In [77]:
import keras
from matplotlib import pyplot as plt
import numpy as np
import gzip
%matplotlib inline
from keras.models import Model
from keras.optimizers import RMSprop
from keras.layers import Input,Dense,Flatten,Dropout,merge,Reshape,Conv2D,MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model,Sequential
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adadelta, RMSprop,SGD,Adam
from keras import regularizers
from keras import backend as K
from keras.utils import to_categorical
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'
from sklearn.preprocessing import OneHotEncoder

In [78]:
minmax = pd.read_csv('C:/Users/brook/Documents/MSDS/MSDS 411/Assignment 3/assign3_minmax.csv')

min_cop = minmax.copy()
y = minmax['credit.class']
minmax = minmax.drop(columns = {"Unnamed: 0"})

minmax['credit.class'] = pd.get_dummies(minmax['credit.class'], drop_first=True)

minmax.head()

,credit.class,checking_status0_X_200,checking_status_200,checking_statusno_checking,duration,credit_historyall_paid,credit_historyexisting_paid,credit_historydelayed_previously,credit_historycritical_other_existing_credit,purposeused_car,...,other_payment_plansstores,other_payment_plansnone,housingown,housingforfree,existing_credits,jobunskilled_resident,jobskilled,jobhigh_qualif_self_emp_mgmt,num_dependents,own_telephoneyes
0,1,0,0,0,0.029412,0,0,0,1,0,...,0,1,1,0,0.333333,0,1,0,0,1
1,0,0,0,0,0.029412,0,0,0,1,0,...,0,1,1,0,0.333333,0,1,0,0,1
2,1,0,0,0,0.029412,0,0,0,1,0,...,0,1,1,0,0.333333,0,1,0,0,1
3,1,0,0,0,0.029412,0,0,0,1,0,...,0,1,1,0,0.333333,0,1,0,0,1
4,0,0,0,0,0.029412,0,0,0,1,0,...,0,1,1,0,0.333333,0,1,0,0,1


In [79]:
from sklearn.model_selection import train_test_split
train_X,valid_X,train_ground,valid_ground = train_test_split(minmax,
 minmax,
 test_size=0.2,
 random_state=13)
train_X.shape

(24799, 45)

In [80]:
import keras
from keras import layers

# This is the size of our encoded representations
encoding_dim = 25  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

input_img = keras.Input(shape =  (45,))
# "encoded" is the encoded representation of the input
encoded = layers.Dense(encoding_dim, activation='relu')(input_img)

encoder = keras.Model(input_img, encoded)
# "decoded" is the lossy reconstruction of the input
decoded = layers.Dense(45, activation='sigmoid')(encoded)

# This model maps an input to its reconstruction
autoencoder = keras.Model(input_img, decoded)

In [81]:
# This is our encoded (32-dimensional) input
encoded_input = keras.Input(shape=(encoding_dim,))
# Retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# Create the decoder model
decoder = keras.Model(encoded_input, decoder_layer(encoded_input))

In [82]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [83]:
autoencoder.fit(train_X, train_X,
                epochs=50,
                batch_size=5000,
                shuffle=True,
                validation_data=(valid_X, valid_X))

Train on 24799 samples, validate on 6200 samples
Epoch 1/50
24799/24799 [==============================] - 1s 36us/step - loss: 0.7023 - val_loss: 0.6933
Epoch 2/50
24799/24799 [==============================] - 0s 6us/step - loss: 0.6875 - val_loss: 0.6792
Epoch 3/50
24799/24799 [==============================] - 0s 4us/step - loss: 0.6737 - val_loss: 0.6659
Epoch 4/50
24799/24799 [==============================] - 0s 5us/step - loss: 0.6603 - val_loss: 0.6524
Epoch 5/50
24799/24799 [==============================] - 0s 5us/step - loss: 0.6465 - val_loss: 0.6382
Epoch 6/50
24799/24799 [==============================] - 0s 7us/step - loss: 0.6318 - val_loss: 0.6228
Epoch 7/50
24799/24799 [==============================] - 0s 7us/step - loss: 0.6158 - val_loss: 0.6060
Epoch 8/50
24799/24799 [==============================] - 0s 8us/step - loss: 0.5984 - val_loss: 0.5879
Epoch 9/50
24799/24799 [==============================] - 0s 6us/step - loss: 0.5797 - val_loss: 0.5686
Epoch 10/50
24

In [84]:
autoencoder = Model(input_img, decoder(encoder(input_img)))
autoencoder.compile(loss='mean_squared_error', optimizer = RMSprop())
autoencoder.summary()

Model: "model_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 45)                0         
_________________________________________________________________
model_20 (Model)             (None, 25)                1150      
_________________________________________________________________
model_22 (Model)             (None, 45)                1170      
Total params: 2,320
Trainable params: 2,320
Non-trainable params: 0
_________________________________________________________________


In [89]:
%load_ext tensorboard
# Clear any logs from previous runs

import tensorflow as tf
import datetime
log_dir = '/tmp/logs_fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [90]:
from keras.callbacks import TensorBoard
# Load the TensorBoard notebook extension

autoencoder.fit(train_X, train_X,
                epochs=10,
                batch_size=128,
                shuffle=True,
                validation_data=(valid_X, valid_X),
                callbacks=[TensorBoard(log_dir='/tmp/logs_fit/')])

Train on 24799 samples, validate on 6200 samples
Epoch 1/10
24799/24799 [==============================] - 1s 26us/step - loss: 3.6372e-05 - val_loss: 2.8964e-05
Epoch 2/10
24799/24799 [==============================] - 1s 22us/step - loss: 3.1777e-05 - val_loss: 2.6760e-05
Epoch 3/10
24799/24799 [==============================] - 1s 21us/step - loss: 2.8170e-05 - val_loss: 2.9420e-05
Epoch 4/10
24799/24799 [==============================] - 1s 21us/step - loss: 2.5083e-05 - val_loss: 1.9910e-05
Epoch 5/10
24799/24799 [==============================] - 1s 23us/step - loss: 2.2533e-05 - val_loss: 2.5066e-05
Epoch 6/10
24799/24799 [==============================] - 0s 20us/step - loss: 2.0855e-05 - val_loss: 2.8124e-05
Epoch 7/10
24799/24799 [==============================] - 1s 30us/step - loss: 1.9032e-05 - val_loss: 2.0052e-05
Epoch 8/10
24799/24799 [==============================] - 1s 20us/step - loss: 1.8195e-05 - val_loss: 1.5990e-05
Epoch 9/10
24799/24799 [=======================

In [91]:
%tensorboard --logdir logs_fit


Reusing TensorBoard on port 6006 (pid 24192), started 0:30:28 ago. (Use '!kill 24192' to kill it.)